In [1]:
import sys
sys.path.append('C:/Users/james/OneDrive/바탕 화면/대학교/수업/2024-1/물류관리/Term_project/Modules')

import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import combinations
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

from utils import *

from savings.link import Link
from savings.node import Node
from savings.graph import Graph
from savings.route import Route
from savings.SavingsModel import SavingsModel


In [2]:
DEPOT_INDEX = 0 # 창고의 data상 index
DEMAND_CV = 0 # 수요 발생의 변동계수(CV)값 설정
DEMAND_COEFFICIENT = 1 # 수요 발생량에 대해 적절히 곱해주기 위함
ROUTE_TYPE = 'trafast' # route 탐색 방식
JSON_PATH = './../../Data/routes_with_depot' # JSON 파일들이 있는 폴더 경로
TRUCK_CAPA = 11560 # 한 트럭이 실을 수 있는 적재 capacity (L)
MAX_TIME = 8 * 60 * 60 * 1000 # ms 단위
SERVICE_TIME = 2*60*1000 # ms 단위

node_data = pd.read_csv('./../../Data/data_with_depot.csv')

In [3]:
# 문제 상황에 대한 Graph 객체 만들기
g = Graph()

# Node 등록
for row in node_data.iterrows():
    data = row[1]

    # Node 생성 후 등록
    n = Node(data['index'], data['longitude'], data['latitude'])
    n.generate_demand(data['demand_mu']*DEMAND_COEFFICIENT, data['capacity'], DEMAND_CV)
    g.add_node(n)

# Link 등록
for n1, n2 in tqdm([comb for comb in combinations(g.nodes, 2)]):
    if n1.index > n2.index:
        temp = n2
        n2 = n1
        n1 = temp

    link_cost, link_time = get_cost_time(n1.index, n2.index, ROUTE_TYPE, JSON_PATH, toll_include=True)
    l = Link(n1, n2, link_cost, link_time)
    g.add_link(l)

  2%|▏         | 44/2145 [00:00<00:10, 208.91it/s]

100%|██████████| 2145/2145 [00:14<00:00, 148.11it/s]


In [65]:
def Clarke_Wright_Savings(route_list, saving_dict, node_list, capa, depot_node, time_constraint, n2n_time, service_time, stochastic_drop=0.0):
    """
    saving_dict: 정렬된 saving의 dictionary 객체
    demand_list: 각 node들의 demand list를 나열 -> depot을 제외한 demand list 만들기
    capa: 트럭의 capacity
    depot_node.index: depot의 index -> 오로지 마지막에 depot index를 붙여줄 때에만 쓰임
    """

    # demand_list 만들기
    demand_list = []
    for node in node_list:
        if node.index == depot_node.index:
            continue
        else:
            demand_list.append(node.demand)

    service_time = [service_time for i in range(len(demand_list))]

    node_state_list = [[0, 0, 0] for i in range(len(demand_list))]
    for link in saving_dict:
        if (np.random.binomial(n=1, p=stochastic_drop, size=1)[0] == 1):
           continue
        if saving_dict[link] < 0:
           continue # saving이 음수인 경우 건너뜀

        # 각 node의 idx
        i_idx = link[0]
        j_idx = link[1]

        # 각 node의 demand
        i_demand = demand_list[i_idx-1]
        j_demand = demand_list[j_idx-1]

        # route_list가 비어있는 경우 -> 새 route 생성
        if (len(route_list) == 0):
            # demand <= capa 확인
            demand = i_demand + j_demand
            if (demand <= capa):
              # time <= time_constraint 확인
              time = n2n_time[0][i_idx] + n2n_time[i_idx][j_idx] + n2n_time[j_idx][0] + service_time[i_idx] + service_time[j_idx]
              if (time <= time_constraint):
                route_list.append([i_idx, j_idx])


        # route_list가 비어있지 않은 경우 -> node_state update
        else :
            for i in range(len(route_list)):
                for j in range(len(route_list[i])):
                    node_state_list[route_list[i][j]-1][0] = 1
                    node_state_list[route_list[i][j]-1][1] = i
                    node_state_list[route_list[i][j]-1][2] = j


            # node_state 가져오기
            i_in = node_state_list[i_idx-1][0]
            i_route_idx = node_state_list[i_idx-1][1]
            i_node_idx = node_state_list[i_idx-1][2]
            j_in = node_state_list[j_idx-1][0]
            j_route_idx = node_state_list[j_idx-1][1]
            j_node_idx = node_state_list[j_idx-1][2]


            # 기존 route들에 i, j 둘 다 없는 경우 -> 새 route 생성
            if (i_in == 0 and j_in == 0):
                # demand <= capa 확인
                demand = i_demand + j_demand
                if (demand <= capa):
                  # time <= time_constraint 확인
                  time = n2n_time[0][i_idx] + n2n_time[i_idx][j_idx] + n2n_time[j_idx][0] + service_time[i_idx-1] + service_time[j_idx-1]
                  if (time <= time_constraint):
                    route_list.append([i_idx, j_idx])


            # 기존 route들에 i, j 둘 다 있는 경우
            elif (i_in == 1 and j_in == 1):
                # i, j 모두 다른 route에 있는 경우
                if (i_route_idx != j_route_idx):
                    # i_route와 j_route의 demand의 합 <= capa인 경우
                    demand = 0
                    for i in route_list[i_route_idx]:
                        demand += demand_list[i-1]
                    for j in route_list[j_route_idx]:
                        demand += demand_list[j-1]

                    # depot -> i_route -> j_route -> depot의 소요 time + 노드의 service time <= time_constraint인 경우
                    time = 0
                    # service time 모두 합하기
                    # 노드 간 이동 시간은 각 branch 별로 merge_route 생성 후, merge_route 통해서 한 번에 구하는게 편할 것 같아서 밑에서 해줄게
                    for i in route_list[i_route_idx]:
                      time += service_time[i-1]
                    for j in route_list[j_route_idx]:
                      time += service_time[j-1]

                    if (demand <= capa):
                        # i, j 모두 route의 첫 번째인 경우
                        if (i_node_idx == 0 and j_node_idx == 0):

                            # merge route
                            merge_route = []
                            for i in route_list[i_route_idx]:
                                merge_route.append(i)
                            for j in route_list[j_route_idx]:
                                merge_route.insert(0, j)

                            # node 간 이동 시간 모두 합하기
                            for i in range(len(merge_route)-1):
                              # merge_route의 i번째 node -> i+1번째 node 이동 시간
                              time += n2n_time[merge_route[i]][merge_route[i+1]]
                              # depot -> 0번째 노드 + 마지막 노드 -> depot 시간 합하기
                              time += n2n_time[0][merge_route[0]]
                              time += n2n_time[merge_route[-1]][0]

                            # time constraint 만족하는지 확인
                            if (time <= time_constraint):
                              # merge_route 추가
                              route_list.append(merge_route)
                              # delete route
                              if (i_route_idx > j_route_idx):
                                  del route_list[i_route_idx]
                                  del route_list[j_route_idx]
                              else :
                                  del route_list[j_route_idx]
                                  del route_list[i_route_idx]

                        # i, j 모두 route의 마지막인 경우
                        elif (i_node_idx == len(route_list[i_route_idx])-1 and j_node_idx == len(route_list[j_route_idx])-1):
                            # merge route
                            merge_route = []
                            for i in route_list[i_route_idx]:
                                merge_route.append(i)
                            for j in route_list[j_route_idx]:
                                merge_route.insert(len(route_list[i_route_idx]), j)

                            # node 간 이동 시간 모두 합하기
                            for i in range(len(merge_route)-1):
                              # merge_route의 i번째 node -> i+1번째 node 이동 시간
                              time += n2n_time[merge_route[i]][merge_route[i+1]]
                              # depot -> 0번째 노드 + 마지막 노드 -> depot 시간 합하기
                              time += n2n_time[0][merge_route[0]]
                              time += n2n_time[merge_route[-1]][0]

                            # time constraint 만족하는지 확인
                            if (time <= time_constraint):
                              # merge_route 추가
                              route_list.append(merge_route)
                              # delete route
                              if (i_route_idx > j_route_idx):
                                  del route_list[i_route_idx]
                                  del route_list[j_route_idx]
                              else :
                                  del route_list[j_route_idx]
                                  del route_list[i_route_idx]


                        # i는 route의 첫 번째, j는 route의 마지막인 경우
                        elif (i_node_idx == 0 and j_node_idx == len(route_list[j_route_idx])-1):
                            # merge route
                            merge_route = []
                            for j in route_list[j_route_idx]:
                                merge_route.append(j)
                            for i in route_list[i_route_idx]:
                                merge_route.append(i)

                            # node 간 이동 시간 모두 합하기
                            for i in range(len(merge_route)-1):
                              # merge_route의 i번째 node -> i+1번째 node 이동 시간
                              time += n2n_time[merge_route[i]][merge_route[i+1]]
                              # depot -> 0번째 노드 + 마지막 노드 -> depot 시간 합하기
                              time += n2n_time[0][merge_route[0]]
                              time += n2n_time[merge_route[-1]][0]

                            # time constraint 만족하는지 확인
                            if (time <= time_constraint):
                              # merge_route 추가
                              route_list.append(merge_route)
                              # delete route
                              if (i_route_idx > j_route_idx):
                                  del route_list[i_route_idx]
                                  del route_list[j_route_idx]
                              else :
                                  del route_list[j_route_idx]
                                  del route_list[i_route_idx]


                        # j는 route의 첫 번째, i는 route의 마지막인 경우
                        elif (j_node_idx == 0 and i_node_idx == len(route_list[i_route_idx])-1):
                            # merge route
                            merge_route = []
                            for i in route_list[i_route_idx]:
                                merge_route.append(i)
                            for j in route_list[j_route_idx]:
                                merge_route.append(j)

                            # node 간 이동 시간 모두 합하기
                            for i in range(len(merge_route)-1):
                              # merge_route의 i번째 node -> i+1번째 node 이동 시간
                              time += n2n_time[merge_route[i]][merge_route[i+1]]
                              # depot -> 0번째 노드 + 마지막 노드 -> depot 시간 합하기
                              time += n2n_time[0][merge_route[0]]
                              time += n2n_time[merge_route[-1]][0]

                            # time constraint 만족하는지 확인
                            if (time <= time_constraint):
                              # merge_route 추가
                              route_list.append(merge_route)
                              # delete route
                              if (i_route_idx > j_route_idx):
                                  del route_list[i_route_idx]
                                  del route_list[j_route_idx]
                              else :
                                  del route_list[j_route_idx]
                                  del route_list[i_route_idx]


            # 기존 route 들에 i, j 중 하나만 있는 경우
            else :
                # i_in == 1 & i가 route의 exterior인 경우
                if (i_in == 1 and (i_node_idx == 0 or i_node_idx == len(route_list[i_route_idx])-1)):
                    # i_route와 j의 demand의 합 <= capa인 경우
                    demand = 0
                    for i in route_list[i_route_idx]:
                        demand += demand_list[i-1]
                    demand += j_demand

                    # depot -> i_route -> j_route -> depot의 소요 time + 노드의 service time <= time_constraint인 경우
                    time = 0
                    # service time 모두 합하기
                    for i in route_list[i_route_idx]:
                      time += service_time[i-1]
                    time += service_time[j_idx-1]

                    if (demand <= capa):
                        # i가 route의 첫 번째인 경우
                        if (i_node_idx == 0):
                          # route_list[i_route_idx] 복사
                          list = []
                          for i in route_list[i_route_idx]:
                            list.append(i)
                          list.insert(0, j_idx)
                          # node 간 이동 시간 모두 합하기
                          for i in range(len(list)-1):
                            time += n2n_time[list[i]][list[i+1]]
                          time += n2n_time[0][list[0]]
                          time += n2n_time[list[-1]][0]

                          # time constraint 만족하는지 확인
                          if (time <= time_constraint):
                            route_list[i_route_idx].insert(0, j_idx)


                        # i가 route의 마지막인 경우
                        else :
                          # route_list[i_route_idx] 복사
                          list = []
                          for i in route_list[i_route_idx]:
                            list.append(i)
                          list.append(j_idx)
                          # node 간 이동 시간 모두 합하기
                          for i in range(len(list)-1):
                            time += n2n_time[list[i]][list[i+1]]
                          time += n2n_time[0][list[0]]
                          time += n2n_time[list[-1]][0]

                          # time constraint 만족하는지 확인
                          if (time <= time_constraint):
                            route_list[i_route_idx].append(j_idx)
                
                # j_in == 1 & j가 route의 exterior인 경우
                elif (j_in == 1 and (j_node_idx == 0 or j_node_idx == len(route_list[j_route_idx])-1)):
                    # j_route와 i의 demand의 합 <= capa인 경우
                    demand = 0
                    for j in route_list[j_route_idx]:
                        demand += demand_list[j-1]
                    demand += i_demand

                    # depot -> i_route -> j_route -> depot의 소요 time + 노드의 service time <= time_constraint인 경우
                    time = 0
                    # service time 모두 합하기
                    for j in route_list[j_route_idx]:
                      time += service_time[j-1]
                    time += service_time[i_idx-1]

                    if (demand <= capa):
                        # j가 route의 첫 번째인 경우
                        if (j_node_idx == 0):
                          # route_list[i_route_idx] 복사
                          list = []
                          for i in route_list[j_route_idx]:
                            list.append(i)
                          list.insert(0, i_idx)
                          # node 간 이동 시간 모두 합하기
                          for i in range(len(list)-1):
                            time += n2n_time[list[i]][list[i+1]]
                          time += n2n_time[0][list[0]]
                          time += n2n_time[list[-1]][0]

                          # time constraint 만족하는지 확인
                          if (time <= time_constraint):
                            route_list[j_route_idx].insert(0, i_idx)



                        # j가 route의 마지막인 경우
                        else :
                          # route_list[j_route_idx] 복사
                          list = []
                          for i in route_list[j_route_idx]:
                            list.append(i)
                          list.append(i_idx)
                          # node 간 이동 시간 모두 합하기
                          for i in range(len(list)-1):
                            time += n2n_time[list[i]][list[i+1]]
                          time += n2n_time[0][list[0]]
                          time += n2n_time[list[-1]][0]

                          # time constraint 만족하는지 확인
                          if (time <= time_constraint):
                            route_list[j_route_idx].append(i_idx)


    # 각 route의 양 쪽에 depot 연결해주기
    for i in route_list:
        i.insert(0, depot_node.index)
        i.append(depot_node.index)

    # route 어디에도 포함 안 된 point 새로운 route로 만들기
    for i in range(len(node_state_list)):
        if (node_state_list[i][0] == 0):
            route_list.append([depot_node.index, i+1, depot_node.index])

    return route_list

In [66]:
svm = SavingsModel(g, DEPOT_INDEX)
svm.calculate_savings()
for s in svm.savings:
    svm.savings[s] += 111821

In [67]:
# 딕셔너리를 리스트로 변환
data_list = [(*key, value) for key, value in svm.savings.items()]

# DataFrame으로 변환
df = pd.DataFrame(data_list, columns=['n1', 'n2', 'saving'])

In [68]:
df[df['saving'] >= 116548].tail(5)

,n1,n2,saving
1080,15,41,116559
1081,33,38,116558
1082,32,40,116553
1083,32,47,116553
1084,35,56,116548


In [69]:
df[(df['n1'] == 35) & (df['n2'] == 56)]

,n1,n2,saving
1084,35,56,116548


In [73]:
df[df['saving']<116548]

,n1,n2,saving
1085,21,56,116547
1086,42,47,116546
1087,1,8,116541
1088,1,40,116541
1089,1,47,116541
...,...,...,...
2075,3,53,108915
2076,5,53,108915
2077,4,53,108907
2078,31,53,108840


In [74]:
temp_savings = {}
for key, val in svm.savings.items():
    n1, n2 = key
    if (n1 == 21) & (n2==56):
        break
    temp_savings[key] = svm.savings[key]
svm.savings = temp_savings
print(len(svm.savings))

1085


In [75]:
# 딕셔너리를 리스트로 변환
data_list = [(*key, value) for key, value in svm.savings.items()]

# DataFrame으로 변환
df2 = pd.DataFrame(data_list, columns=['n1', 'n2', 'saving'])
df2.tail(5)

,n1,n2,saving
1080,15,41,4738
1081,33,38,4737
1082,32,40,4732
1083,32,47,4732
1084,35,56,4727


In [76]:
#svm.calculate_savings()
svm.apply_algorithm([], TRUCK_CAPA, Clarke_Wright_Savings, MAX_TIME, SERVICE_TIME, stochastic_drop=0.0)
print(len(svm.savings))

1085


In [77]:
for r in svm.routes:
    print(r)

Route : 0 - 53 - 50 - 12 - 8 - 41 - 47 - 40 - 48 - 17 - 37 - 10 - 52 - 0
Demand met : 686.5508541700001, Cost : 28183, Time 5H 5M
Route : 0 - 18 - 28 - 38 - 0
Demand met : 214.12342718, Cost : 34230, Time 3H 52M
Route : 0 - 31 - 45 - 1 - 15 - 14 - 19 - 23 - 42 - 54 - 32 - 39 - 22 - 29 - 30 - 20 - 27 - 25 - 0
Demand met : 907.137935241, Cost : 24895, Time 4H 54M
Route : 0 - 11 - 6 - 9 - 16 - 44 - 2 - 0
Demand met : 370.598431744, Cost : 18946, Time 2H 38M
Route : 0 - 4 - 5 - 3 - 0
Demand met : 151.75062185000002, Cost : 15076, Time 1H 53M
Route : 0 - 13 - 33 - 24 - 26 - 43 - 49 - 0
Demand met : 305.83233445, Cost : 14428, Time 2H 16M
Route : 0 - 46 - 7 - 51 - 0
Demand met : 22.510921447999998, Cost : 12210, Time 1H 42M
Route : 0 - 59 - 64 - 61 - 65 - 58 - 55 - 57 - 60 - 63 - 62 - 56 - 0
Demand met : 2242.0504826, Cost : 11492, Time 3H 41M
Route : 0 - 35 - 21 - 36 - 34 - 0
Demand met : 16.970410184, Cost : 9355, Time 1H 29M


In [78]:
print(Route(g, [0, 59, 64, 61, 65, 58, 55, 57, 60, 63, 62, 56, 35, 21, 36, 34, 0]))

Route : 0 - 59 - 64 - 61 - 65 - 58 - 55 - 57 - 60 - 63 - 62 - 56 - 35 - 21 - 36 - 34 - 0
Demand met : 2259.020892784, Cost : 16120, Time 4H 21M


In [52]:
print(g.get_link(0, 35).cost)
print(g.get_link(0, 56).cost)
print(g.get_link(56, 35).cost)

4529
3705
3507
